# Drill Holes with baselode.drill

Basic operations

* Load collars, surveys, assays using the library loaders
* Desurvey and attach assay positions
* Visualize a single-variable strip log

## Install baselode

Into your venv or conda-env:

1. From pypi

`pip install baselode`

2. From a local build

`cd python/src`
`python -m build`
`pip install baselode<ver>.whl`

3. Locally in dev mode

`python -m pip install -e python/src`

In [ ]:
# Uncomment to install locally in dev mode
! pwd
! python -m pip install -e python/src

In [ ]:
import pathlib
import pandas as pd

import baselode.drill
import baselode.drill.data
import baselode.drill.view
import baselode.map

## Access test data csvs

This data is from the GSWA open file geochemistry dataset.

>> attribution

In [ ]:
repo_root = pathlib.Path(".").resolve()
TEST_DIR = repo_root.joinpath('test/data/gswa')

COLLAR_CSV = TEST_DIR.joinpath("gswa_sample_collars.csv")
SURVEY_CSV = TEST_DIR.joinpath("gswa_sample_survey.csv")
ASSAY_FLAT_CSV = TEST_DIR.joinpath("gswa_sample_assays.csv")

# use this to load collars. It will select CompanyHoleId as the hole_id used elsewhere in the code
col_map = { "CompanyHoleId": "hole_id" }

### Collars

In [ ]:
collar_gdf = baselode.drill.data.load_collars(COLLAR_CSV, source_column_map=col_map, keep_all=True)
display(collar_gdf.head())

collar_gdf = baselode.drill.data.load_collars(COLLAR_CSV, source_column_map=col_map, keep_all=False)
display(collar_gdf.head())

collar_gdf.crs

### Survey

In [ ]:
survey_df = baselode.drill.data.load_surveys(SURVEY_CSV, source_column_map=col_map)
display(survey_df.head())

survey_df = baselode.drill.data.load_surveys(SURVEY_CSV, source_column_map=col_map, keep_all=False)
display(survey_df.head())

### Assays

In [ ]:
assay_df = baselode.drill.data.load_assays(ASSAY_FLAT_CSV, source_column_map=col_map, keep_all=False)
display(assay_df.head())

assay_df = baselode.drill.data.load_assays(ASSAY_FLAT_CSV, source_column_map=col_map)
display(assay_df.head())



In [ ]:
import baselode.drill.desurvey
import baselode.drill.view


traces = baselode.drill.desurvey.build_traces(collars=collar_gdf, surveys=survey_df, step=5.0,)
assays_with_xyz = baselode.drill.desurvey.attach_assay_positions(assays=assay_df, traces=traces)

display(assays_with_xyz)

In [ ]:
from baselode.datamodel import HOLE_ID

hole_id = assays_with_xyz[HOLE_ID].iloc[10]
print(hole_id)
df_plot = assays_with_xyz[assays_with_xyz[HOLE_ID] == hole_id]
target_col = "cr_ppm"

with pd.option_context("display.max_columns", None):
    # show which columns have non-null values for this hole_id
    non_null_cols = df_plot.columns[df_plot.notna().any()]
    display(df_plot[non_null_cols])

fig = baselode.drill.view.plot_drillhole_trace(df=df_plot, value_col=target_col, chart_type="markers+line", use_mid=True)
fig.update_layout(width=220, height=600)
fig.show()